<a href="https://colab.research.google.com/github/Urbanus-Kathitu/Cora-Moringa-School-FAQ-Chatbot-Group-1/blob/main/test_chatbotfi_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing the necessary libraries
!pip install nltk
!pip install tflearn
import numpy
import tflearn
import tensorflow
import random
import json
import nltk
nltk.download('punkt')
from tensorflow.python.framework import ops
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=2d3405deb2596304d0db30a01522c3a7b6f7d05036ebf65d5c11a24b4893ddca
  Stored in directory: /root/.cache/pip/wheels/4a/d5/f8/9585b4a100c0fd73da204ee785457d67c85e1b9050f009a849
Successfully built tflearn


Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#Loading the Json file with the intents,patterns and responses
import json
with open('/content/FAQs') as file:

    data = json.load(file)
print (data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'accommodation', 'patterns': ['do you offer accommodation', 'are there boarding facilities?', 'do you have hostels ?'], 'responses': ['No, we currently don’t provide accommodation. However, there are a number of private hostels along Ngong road, which are close to the school.'], 'context_set': ''}, {'tag': 'remote', 'patterns': ['Do you offer remote learning courses?', 'Are there online classes?', 'Can i take the course online?', 'Can i take virtual classes?', 'Do you offer part time classes?'], 'responses': ['Moringa School’s remote classes are done live and online; wi

In [ ]:
#Extracting Data
#For each pattern we will turn it into a list of words using nltk.word_tokenizer. 
#We will then add each pattern into our docs_x list and its associated tag into the docs_y list
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [ ]:
#Stemming our words
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

#create a unique list of stemmed word
labels = sorted(labels)

In [ ]:
#Coverting our sentencing to a bag of words.
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [ ]:
#Vconvert our training data and output to numpy arraystraining = numpy.array(training)
output = numpy.array(output)

In [ ]:
#defining the architecture of our model
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
#Training the Model

model.fit(training, output, n_epoch=1000, batch_size=5, show_metric=True)


Training Step: 10999  | total loss: 0.52293 | time: 0.060s
| Adam | epoch: 1000 | loss: 0.52293 - acc: 0.9715 -- iter: 50/55
Training Step: 11000  | total loss: 0.47212 | time: 0.065s
| Adam | epoch: 1000 | loss: 0.47212 - acc: 0.9743 -- iter: 55/55
--


In [ ]:
#Creating a function to tokenize, stem and bag the input
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

#Creating a funtion for taking inputs from the user,applying the model to it and printing out a random response from the tag with the highest probability
def chat():
    print("Hello, I am Kora,the moringa school FAQ bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        if results[results_index] >0.5:
            for tg in data["intents"]:
              if tg['tag'] == tag:
                  responses = tg['responses']

            print(random.choice(responses))
        else:
          print('I did not understand, kindly ask another question')

chat()

Hello, I am Kora,the moringa school FAQ bot (type quit to stop)!
You: hi
Hello!
You: whats my name
I did not understand, kindly ask another question
You: do you offer accomodation
No, we currently don’t provide accommodation. However, there are a number of private hostels along Ngong road, which are close to the school.
You: how much do i pay
Yes, students that have enrolled to our remote classes are provided with data bundles
You: which courses are offered at moringa
No, we do not offer installment plans for the Prep course but we do for the core course


KeyboardInterrupt: ignored